In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.sql.functions import col, when, avg, to_timestamp
import os

try:
    spark
except NameError:
    spark = SparkSession.builder \
        .appName("IoT ETL Pipeline") \
        .config("spark.driver.extraClassPath", "/home/jovyan/work/Graduation/mssql-jdbc-13.2.1.jre8.jar") \
        .getOrCreate()

schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("device_id", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("humidity", IntegerType(), True),
    StructField("pressure", IntegerType(), True),
    StructField("air_quality", IntegerType(), True),
    StructField("noise_level", IntegerType(), True),
    StructField("battery_level", IntegerType(), True),
    StructField("signal_strength", IntegerType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("raw_message", StringType(), True)
])

data_folder = "/home/jovyan/work/Graduation/_IOT_Batches"

df = spark.read.option("header", "true").schema(schema).csv(os.path.join(data_folder, "*.csv"))

print("N.rows:", df.count())
print("N.col:", len(df.columns))

df = df.withColumn("timestamp", to_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))

df = df.withColumn(
    "temperature_status",
    when(col("temperature") > 30, "High")
    .when(col("temperature") < 22, "Low")
    .otherwise("Normal")
)

df = df.withColumn(
    "battery_status",
    when(col("battery_level") < 20, "Low Battery")
    .otherwise("OK")
)

if "temperature" in df.columns:
    df = df.filter((col("temperature") >= -10) & (col("temperature") <= 60))

df = df.withColumn(
    "anomaly_flag",
    when((col("temperature") > 40) | (col("temperature") < 10), 1).otherwise(0)
)

agg_df = df.groupBy("device_id").agg(
    avg("temperature").alias("avg_temp"),
    avg("humidity").alias("avg_humidity"),
    avg("pressure").alias("avg_pressure"),
    avg("air_quality").alias("avg_air_quality"),
    avg("noise_level").alias("avg_noise_level"),
    avg("battery_level").alias("avg_battery_level"),
    avg("signal_strength").alias("avg_signal_strength")
)

print("Displaying 10 rows:")
agg_df.show(10)

jdbc_url = "jdbc:sqlserver://host.docker.internal:1433;databaseName=IoT_DB;encrypt=false;"

connection_properties = {
    "user": "sa",
    "password": "Salma_SQL@2005DEPI",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

df.write.jdbc(url=jdbc_url, table="iot_data", mode="overwrite", properties=connection_properties)
agg_df.write.jdbc(url=jdbc_url, table="iot_summary", mode="overwrite", properties=connection_properties)

print("Done")

spark.stop()


N.rows: 3419
N.col: 12
Displaying 10 rows:
+---------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+
|device_id|          avg_temp|      avg_humidity|      avg_pressure|  avg_air_quality|   avg_noise_level| avg_battery_level|avg_signal_strength|
+---------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+
| sensor_1|27.580568011958153| 55.14648729446936|1014.2739520958083|51.64221556886228| 56.81437125748503| 54.15119760479042| -60.17365269461078|
| sensor_4|27.618080808080816| 55.86147186147186|1013.6994219653179|50.02167630057804|56.372832369942195| 53.74710982658959| -59.40462427745665|
| sensor_2|27.533811659192825| 53.69506726457399|1015.7563527653214|51.34977578475336|54.478325859491775|53.926756352765324|-60.663677130044846|
| sensor_3|27.470942562592032|54.166421207658324|1014.7085798816568|51.73224852071006| 